# Part 1: Collecting the Twitter data


In [1]:
""" Libraries """
import csv
import pandas as pd
from getpass import getpass
from time import sleep
import datetime
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
# chrome browser
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

In [2]:
def get_tweet_data(card):
    """Extract data from tweet data"""
    
    username = card.find_element(by=By.XPATH, value='.//div[2]/div[2]/div[1]/div[1]//span').text
    handle = card.find_element(by=By.XPATH,value='.//span[contains(text(), "@")]').text
    comment = card.find_element(by=By.XPATH, value='.//div[2]/div[2]/div[2]/div[1]').text
    responding = card.find_element(by=By.XPATH, value='.//div[2]/div[2]/div[2]/div[2]').text
    text = comment+responding
    reply = card.find_element(by=By.XPATH, value='.//div[@data-testid="reply"]').text
    retweet = card.find_element(by=By.XPATH, value='.//div[@data-testid="retweet"]').text
    like = card.find_element(by=By.XPATH, value='.//div[@data-testid="like"]').text
    try:
        postdate = card.find_element(by=By.XPATH, value='.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    tweet = (username,handle,postdate,comment,responding,text,reply,retweet,like)
    return tweet

In [3]:
#create instance of web-driver
driver = Chrome("C:/Program Files/chromedriver.exe")

C:\Users\Linds\AppData\Local\Temp\ipykernel_10292\4099496780.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("C:/Program Files/chromedriver.exe")


In [4]:
#navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()

In [5]:
username = driver.find_element(by=By.XPATH, value='//input[@name="text"]')
username.send_keys("username") #due to privacy reasons, username is not shown
username.send_keys(Keys.RETURN)

In [6]:
password = driver.find_element(by=By.XPATH, value='//input[@name="password"]')
password.send_keys("password") #due to privacy reasons, password is not shown
password.send_keys(Keys.RETURN)

In [18]:
#find search input and search for term
search_input = driver.find_element(by=By.XPATH, value='//input[@aria-label="Search query"]')
search_input.send_keys('#savetherhino until:2021-12-31 since:2019-01-01')
search_input.send_keys(Keys.RETURN)

#### Hashtags
1. wildlifecrime Rhino
2. Rhino Poaching
3. Rhinoceros Poaching
4. Rhinopoaching
5. Antipoaching Rhino
6. Rhinohorn
7. Savetherhino

This process is done for all the different hashtags. The datasets are combined into one file. 

In [19]:
#navigate to historical 'latest' tab
driver.find_element("link text", "Latest").click()

In [20]:
#get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(by=By.XPATH, value='//article[@data-testid="tweet"]')
    for card in page_cards:
        try: 
            tweet = get_tweet_data(card)
        except Exception as e:
            print("This tweet data is not found")
            continue
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:
        #check scroll position   
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            #end scroll:
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) #attempt another scroll
        else:
            last_position = curr_position
            break

In [22]:
#saving the data:
with open('Tweets/rhino_tweets_#savetherhino.csv','w',newline='',encoding='utf-8') as f:
    header = ['Username', 'Handle', 'Timestamp', 'Comments', 'Responding', 'Text', 
              'Reply', 'Retweet', 'Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)